In [ ]:
# Generate a Spark Session

from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from Sockets')
    .master("local[*]")
    .getOrCreate()
)

spark

In [ ]:
# Read input data

# df_raw = spark.read.format("text").load("example.txt") # batch processing code

# streaming version
df_raw = spark.readStream.format("socket").option("host","localhost").option("port", "9999").load()

In [ ]:
df_raw.printSchema()

#### Show commands ".show()" are not required in the case of streaming source.

In [ ]:
# Split the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split("value", " "))

In [ ]:
df_words.show() 

In [ ]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")

In [ ]:
df_explode.show() 

In [ ]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [ ]:
df_agg.show()

In [ ]:
# Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()